In [1]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os

base_path = "/content/drive/MyDrive/SHL Task/dataset"
print(os.listdir(base_path))


['test.csv', 'train.csv', 'sample_submission.csv', 'audios_train', 'audios_test', 'shl-grammar-scoring.ipynb']


In [ ]:
!pip install -q openai-whisper nltk scikit-learn pandas tqdm librosa


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 13.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
print("CUDA available:", torch.cuda.is_available())
print("GPU:", torch.cuda.get_device_name(0))


CUDA available: True
GPU: Tesla T4


In [ ]:
base_path = "/content/drive/MyDrive/SHL Task/dataset"
train_df = pd.read_csv(os.path.join(base_path, "train.csv"))
test_df = pd.read_csv(os.path.join(base_path, "test.csv"))

print(train_df.shape, test_df.shape)
train_df.head()


(444, 2) (195, 1)


,filename,label
0,audio_1261.wav,1.0
1,audio_942.wav,1.5
2,audio_1110.wav,1.5
3,audio_1024.wav,1.5
4,audio_538.wav,2.0


In [ ]:
import os
import whisper
import pandas as pd
import nltk
from nltk import word_tokenize, pos_tag
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from tqdm import tqdm
import numpy as np

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [ ]:
base_path = "/content/drive/MyDrive/SHL Task/dataset"

train_df = pd.read_csv(os.path.join(base_path, "train.csv"))
test_df = pd.read_csv(os.path.join(base_path, "test.csv"))

print(train_df.shape, test_df.shape)
train_df.head()


(444, 2) (195, 1)


,filename,label
0,audio_1261.wav,1.0
1,audio_942.wav,1.5
2,audio_1110.wav,1.5
3,audio_1024.wav,1.5
4,audio_538.wav,2.0


In [ ]:
model = whisper.load_model("base")  # Whisper-base works great with T4 GPU


100%|████████████████████████████████████████| 139M/139M [00:00<00:00, 169MiB/s]


In [ ]:
def extract_features(audio_path):
    # Transcribe speech to text
    result = model.transcribe(audio_path, fp16=False)
    text = result['text'].strip()

    # Tokenization + POS tagging
    tokens = word_tokenize(text)
    pos_tags = pos_tag(tokens)

    # Rule-based grammar features
    total_words = len(tokens)
    num_nouns = sum(1 for w, t in pos_tags if t.startswith('NN'))
    num_verbs = sum(1 for w, t in pos_tags if t.startswith('VB'))
    num_adjs = sum(1 for w, t in pos_tags if t.startswith('JJ'))
    avg_word_len = np.mean([len(w) for w in tokens]) if tokens else 0

    return {
        'text': text,
        'total_words': total_words,
        'num_nouns': num_nouns,
        'num_verbs': num_verbs,
        'num_adjs': num_adjs,
        'avg_word_len': avg_word_len
    }


In [ ]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')   # ✅ NEW fix for NLTK 3.9+
nltk.download('averaged_perceptron_tagger')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [ ]:
import os
import whisper
import pandas as pd
import nltk
from nltk import word_tokenize, pos_tag
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from tqdm import tqdm
import numpy as np

# 🧠 Download all updated NLTK models
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger')
nltk.download('averaged_perceptron_tagger_eng')  # 🩵 New tagger fix


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.


True

In [ ]:
import os
import pandas as pd
from tqdm import tqdm

# path setup
base_path = "/content/drive/MyDrive/SHL Task/dataset"
progress_path = os.path.join(base_path, "train_progress.csv")

# Load existing progress if available
if os.path.exists(progress_path):
    train_features = pd.read_csv(progress_path).to_dict('records')
    start_index = len(train_features)
    print(f"✅ Resuming from {start_index} already processed files.")
else:
    train_features = []
    start_index = 0
    print("🚀 Starting fresh feature extraction...")

save_every = 20  # save progress every 20 audios

for i, row in tqdm(train_df.iloc[start_index:].iterrows(), total=len(train_df) - start_index):
    audio_path = os.path.join(base_path, "audios_train", row['filename'])
    try:
        feats = extract_features(audio_path)
        feats['label'] = row['label']
        train_features.append(feats)
    except Exception as e:
        print(f"❌ Error on {row['filename']}: {e}")

    # auto-save every 20 iterations
    if (i + 1) % save_every == 0 or (i + 1) == len(train_df):
        pd.DataFrame(train_features).to_csv(progress_path, index=False)
        print(f"💾 Saved progress at {i + 1 + start_index} files.")


🚀 Starting fresh feature extraction...


  5%|▍         | 20/444 [01:36<21:24,  3.03s/it]

💾 Saved progress at 20 files.


  9%|▉         | 40/444 [02:12<16:32,  2.46s/it]

💾 Saved progress at 40 files.


...

💾 Saved progress at 420 files.


 99%|█████████▉| 440/444 [25:00<00:24,  6.10s/it]

💾 Saved progress at 440 files.


100%|██████████| 444/444 [25:15<00:00,  3.41s/it]

💾 Saved progress at 444 files.


In [ ]:
import pandas as pd
base_path = "/content/drive/MyDrive/SHL Task/dataset"

train_feat_df = pd.read_csv(os.path.join(base_path, "train_progress.csv"))
print(train_feat_df.shape)
train_feat_df.head()


(444, 7)


,text,total_words,num_nouns,num_verbs,num_adjs,avg_word_len,label
0,nd my favourite hobbies cultivation of plants ...,36,12,3,3,4.666667,1.0
1,The playground looks like very clear and neat ...,45,8,7,4,4.000000,1.5
2,My Girl is to become an electrical employee. A...,29,10,5,3,5.103448,1.5
3,My favorite place is in Andhra Pradesh. It is ...,76,24,11,3,3.789474,1.5
4,"My favorite places, my favorite places, Mutti ...",128,32,11,12,3.734375,2.0


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import joblib

X = train_feat_df[['total_words', 'num_nouns', 'num_verbs', 'num_adjs', 'avg_word_len']]
y = train_feat_df['label']

rf_model = RandomForestRegressor(n_estimators=300, max_depth=10, random_state=42)
rf_model.fit(X, y)

# Evaluate on train set
y_pred = rf_model.predict(X)
mse = mean_squared_error(y, y_pred)
r2 = r2_score(y, y_pred)

print(f"✅ Model trained successfully!")
print(f"Train MSE: {mse:.4f}")
print(f"Train R² Score: {r2:.4f}")

# Save the model
joblib.dump(rf_model, os.path.join(base_path, "grammar_rf_model.pkl"))


✅ Model trained successfully!
Train MSE: 0.3405
Train R² Score: 0.7251


['/content/drive/MyDrive/SHL Task/dataset/grammar_rf_model.pkl']

In [ ]:
import os
import pandas as pd
from tqdm import tqdm

base_path = "/content/drive/MyDrive/SHL Task/dataset"
progress_path_test = os.path.join(base_path, "test_progress.csv")

print("🔍 Checking if previous test extraction exists...")

# If some progress already exists
if os.path.exists(progress_path_test):
    test_features = pd.read_csv(progress_path_test).to_dict('records')
    start_index = len(test_features)
    print(f"✅ Resuming from {start_index} already processed test files.")
else:
    test_features = []
    start_index = 0
    print("🚀 Starting fresh extraction for test set...")

save_every = 10  # save progress every 10 files

for i, row in tqdm(test_df.iloc[start_index:].iterrows(), total=len(test_df) - start_index):
    audio_path = os.path.join(base_path, "audios_test", row['filename'])
    try:
        feats = extract_features(audio_path)
        test_features.append(feats)
    except Exception as e:
        print(f"❌ Error on {row['filename']}: {e}")

    # Auto-save progress every 10 files
    if (i + 1) % save_every == 0 or (i + 1) == len(test_df):
        pd.DataFrame(test_features).to_csv(progress_path_test, index=False)
        print(f"💾 Saved test progress at {i + 1 + start_index} files.")


🔍 Checking if previous test extraction exists...
🚀 Starting fresh extraction for test set...


  5%|▌         | 10/195 [00:23<05:25,  1.76s/it]

💾 Saved test progress at 10 files.


 10%|█         | 20/195 [00:50<08:59,  3.08s/it]

💾 Saved test progress at 20 files.


...

💾 Saved test progress at 180 files.


 97%|█████████▋| 190/195 [08:48<00:25,  5.02s/it]

💾 Saved test progress at 190 files.


100%|██████████| 195/195 [09:31<00:00,  2.93s/it]

💾 Saved test progress at 195 files.


In [ ]:
test_feat_df = pd.read_csv(os.path.join(base_path, "test_progress.csv"))
print("✅ Test features loaded successfully!")
print("Shape:", test_feat_df.shape)
test_feat_df.head()


✅ Test features loaded successfully!
Shape: (195, 6)


,text,total_words,num_nouns,num_verbs,num_adjs,avg_word_len
0,pr Miguel nameμα ま lei stood up,7,5,1,0,3.571429
1,my hobbies are playing cricket because I am a ...,60,14,14,0,3.900000
2,At this market you can find a lot of things. A...,143,32,21,4,3.538462
3,My goal is I become an interpreter in the prem...,78,19,16,3,4.012821
4,"Okay, in the market, there is a lot of people,...",127,28,19,9,3.606299


In [ ]:
import joblib
import os
import pandas as pd

base_path = "/content/drive/MyDrive/SHL Task/dataset"

# Load your trained Random Forest model
rf_model = joblib.load(os.path.join(base_path, "grammar_rf_model.pkl"))

# Prepare test features
X_test = test_feat_df[['total_words', 'num_nouns', 'num_verbs', 'num_adjs', 'avg_word_len']]

# Predict grammar scores
preds = rf_model.predict(X_test)

# Create submission DataFrame
submission = pd.DataFrame({
    'filename': test_df['filename'],
    'label': preds
})

# Save the submission file
submission_path = os.path.join(base_path, "submission.csv")
submission.to_csv(submission_path, index=False)

print("✅ Final submission file saved successfully!")
print("📂 Path:", submission_path)
submission.head()


✅ Final submission file saved successfully!
📂 Path: /content/drive/MyDrive/SHL Task/dataset/submission.csv


,filename,label
0,audio_706.wav,4.274238
1,audio_800.wav,3.739299
2,audio_68.wav,3.156359
3,audio_1267.wav,3.586531
4,audio_683.wav,3.228601


In [ ]:
!pip install gradio whisper nltk joblib librosa


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for whisper: filename=whisper-1.1.10-py3-none-any.whl size=41120 sha256=721adc0b6aabba2df5e5621d4ef33c561e2ba49bcef054d1d3a57aef2b4fbf46
  Stored in directory: /root/.cache/pip/wheels/34/b8/4e/9c4c3351d670e06746a340fb4b7d854c76517eec225e5b32b1
Successfully built whisper


In [2]:
!pip install gradio whisper nltk joblib librosa soundfile


In [4]:
!pip install gradio whisper nltk joblib soundfile


In [ ]:
import gradio as gr
import whisper
import nltk
import joblib
import librosa
import numpy as np
import pandas as pd
from nltk import word_tokenize, pos_tag
import tempfile
import os

# -----------------------------
# 1️⃣ Setup and Load Resources
# -----------------------------
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

# Load Whisper model (base version is fast and free)
asr_model = whisper.load_model("base")

# Load your trained RandomForest model
model_path = "/content/drive/MyDrive/SHL Task/dataset/grammar_rf_model.pkl"
rf_model = joblib.load(model_path)

# -----------------------------
# 2️⃣ Extract Linguistic Features
# -----------------------------
def extract_features_from_audio(audio_path):
    """Transcribe audio and extract linguistic features"""
    result = asr_model.transcribe(audio_path, fp16=False)
    text = result['text'].strip()

    tokens = word_tokenize(text)
    pos_tags = pos_tag(tokens)

    total_words = len(tokens)
    num_nouns = sum(1 for w, t in pos_tags if t.startswith('NN'))
    num_verbs = sum(1 for w, t in pos_tags if t.startswith('VB'))
    num_adjs = sum(1 for w, t in pos_tags if t.startswith('JJ'))
    avg_word_len = np.mean([len(w) for w in tokens]) if total_words > 0 else 0

    features = {
        "total_words": total_words,
        "num_nouns": num_nouns,
        "num_verbs": num_verbs,
        "num_adjs": num_adjs,
        "avg_word_len": avg_word_len
    }

    return text, features

# -----------------------------
# 3️⃣ Prediction Function
# -----------------------------
def predict_grammar_score(audio):
    """Main pipeline for Gradio — audio input to score"""
    if audio is None:
        return "Please upload an audio file!", None, None

    # Save temp audio file
    with tempfile.NamedTemporaryFile(delete=False, suffix=".wav") as tmp:
        librosa.output.write_wav(tmp.name, audio[1], sr=audio[0])
        audio_path = tmp.name

    # Extract text + features
    text, features = extract_features_from_audio(audio_path)
    X = pd.DataFrame([features])

    # Predict grammar score
    score = rf_model.predict(X)[0]
    feature_summary = "\n".join([f"{k}: {v}" for k, v in features.items()])

    return text, feature_summary, f"{score:.2f} / 5.0"

# -----------------------------
# 4️⃣ Gradio Interface
# -----------------------------
interface = gr.Interface(
    fn=predict_grammar_score,
    inputs=gr.Audio(type="numpy", label="🎤 Upload or Record English Speech (WAV/MP3)"),
    outputs=[
        gr.Textbox(label="🗣️ Transcription"),
        gr.Textbox(label="📊 Linguistic Features"),
        gr.Textbox(label="🧮 Predicted Grammar Score (0–5 Scale)")
    ],
    title="🎙️ Grammar Scoring Engine — Satyam’s SHL Project",
    description="Upload or record English speech. This app transcribes audio, extracts linguistic features (POS-based), and predicts a grammar score using a Random Forest model trained on SHL dataset.",
    theme="default"
)

if __name__ == "__main__":
    interface.launch(debug=True)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://0e82475e7d2b6a32ad.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/gradio/queueing.py", line 759, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/gradio/route_utils.py", line 354, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/gradio/blocks.py", line 2116, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/gradio/blocks.py", line 1623, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/anyio/to_thread.py", line 56, in run_sync
    return await get_async_backend().run_sync_in_worker_thread(
           ^^^^^

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://0e82475e7d2b6a32ad.gradio.live
